<a href="https://colab.research.google.com/github/Tony14100/Date-cocktail/blob/main/%D0%A2%D0%B8%D1%82%D0%B0%D0%BD%D0%B8%D0%BA_%D1%82%D1%8E%D0%BD%D0%B5%D1%80_%D0%A4%D0%BE%D1%80%D0%B5%D1%81%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow_decision_forests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.0 MB/s eta 0:00:00


In [ ]:
pip install wurlitzer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow_decision_forests as tfdf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

try:
  from wurlitzer import sys_pipes
except:
  from colabtools.googlelog import CaptureLog as sys_pipes

from IPython.core.magic import register_line_magic
from IPython.display import Javascript

In [ ]:
dataset = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dataset.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [ ]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

434 examples in training, 193 examples for testing.


In [ ]:
test_ds_pd

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
8,1,female,4.0,1,1,16.7000,Third,G,Southampton,n
13,1,male,28.0,0,0,13.0000,Second,unknown,Southampton,y
19,0,male,28.0,0,0,7.2250,Third,unknown,Cherbourg,y
...,...,...,...,...,...,...,...,...,...,...
611,1,male,4.0,1,1,11.1333,Third,unknown,Southampton,n
617,1,female,15.0,0,0,7.2250,Third,unknown,Cherbourg,y
622,0,male,28.0,0,0,10.5000,Second,unknown,Southampton,y
624,1,female,19.0,0,0,30.0000,First,B,Southampton,y


In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label="survived")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label="survived")

In [ ]:
# Install the Keras tuner
!pip install keras-tuner -U -qq
import keras_tuner as kt

In [ ]:
!%%time

def build_model(hp):
  """Creates a model."""

  model = tfdf.keras.GradientBoostedTreesModel(
      min_examples=hp.Choice("min_examples", [2, 5, 7, 10]),
      categorical_algorithm=hp.Choice("categorical_algorithm", ["CART", "RANDOM"]),
      max_depth=hp.Choice("max_depth", [4, 5, 6, 7]),
      # The keras tuner convert automaticall boolean parameters to integers.
      use_hessian_gain=bool(hp.Choice("use_hessian_gain", [True, False])),
      shrinkage=hp.Choice("shrinkage", [0.02, 0.05, 0.10, 0.15]),
      num_candidate_attributes_ratio=hp.Choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0]),
  )

  # Optimize the model accuracy as computed on the validation dataset.
  model.compile(metrics=["accuracy"])
  return model

keras_tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=50,
    overwrite=True,
    directory="/tmp/keras_tuning")

# Important: The tuning should not be done on the test dataset.

# Extract a validation dataset from the training dataset. The new training
# dataset is called the "sub-training-dataset".

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

sub_train_df, sub_valid_df = split_dataset(train_ds_pd)
sub_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(sub_train_df, label="survived")
sub_valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(sub_valid_df, label="survived")

# Tune the model
keras_tuner.search(sub_train_ds, validation_data=sub_valid_ds)

Trial 50 Complete [00h 00m 02s]
val_accuracy: 0.8169013857841492

Best val_accuracy So Far: 0.8309859037399292
Total elapsed time: 00h 01m 29s


In [ ]:
# Tune the model
best_hyper_parameters = keras_tuner.get_best_hyperparameters()[0].values
print("Best hyper-parameters:", keras_tuner.get_best_hyperparameters()[0].values)

Best hyper-parameters: {'min_examples': 7, 'categorical_algorithm': 'CART', 'max_depth': 7, 'use_hessian_gain': 0, 'shrinkage': 0.15, 'num_candidate_attributes_ratio': 1.0}


In [ ]:
!%set_cell_height 300
# Train the model
# The keras tuner convert automaticall boolean parameters to integers.
best_hyper_parameters["use_hessian_gain"] = bool(best_hyper_parameters["use_hessian_gain"])
best_model = tfdf.keras.GradientBoostedTreesModel(**best_hyper_parameters)
best_model.fit(train_ds, verbose=2)

/bin/bash: line 0: fg: no job control
Use /tmp/tmpttmgjgiw as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'sex': <tf.Tensor 'data:0' shape=(None,) dtype=string>, 'age': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'n_siblings_spouses': <tf.Tensor 'data_2:0' shape=(None,) dtype=int64>, 'parch': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>, 'fare': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'class': <tf.Tensor 'data_5:0' shape=(None,) dtype=string>, 'deck': <tf.Tensor 'data_6:0' shape=(None,) dtype=string>, 'embark_town': <tf.Tensor 'data_7:0' shape=(None,) dtype=string>, 'alone': <tf.Tensor 'data_8:0' shape=(None,) dtype=string>}
Label: Tensor("data_9:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'sex': SemanticTensor(semantic=<Semantic.CATEGORICAL: 2>, tensor=<tf.Tensor 'data:0' shape=(None,) dtype=string>), 'age': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:

[INFO 23-04-23 17:03:21.0137 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-04-23 17:03:21.0137 UTC kernel.cc:774] Collect training examples
[INFO 23-04-23 17:03:21.0138 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-04-23 17:03:21.0139 UTC kernel.cc:393] Number of batches: 1
[INFO 23-04-23 17:03:21.0139 UTC kernel.cc:394] Number of examples: 434
[INFO 23-04-23 17:03:21.0141 UTC data_spec_inference.cc:305] 2 item(s) have been pruned (i.e. they are considered out of dictionary) for the column deck (6 item(s) left) because min_value_count=5 and max_number_of_unique_values=2000
[INFO 23-04-23 17:03:21.0143 UTC kernel.cc:794] Train

Model trained in 0:00:00.752826
Compiling model...
Model compiled.


In [ ]:
# Evaluate the model
best_model.compile(["accuracy"])
tuned_test_accuracy = best_model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy with the Keras Tuner: {tuned_test_accuracy:.4f}")

Test accuracy with the Keras Tuner: 0.7358


In [ ]:
predictions = best_model.predict(test_ds)

1/1 [==============================] - 0s 55ms/step


In [ ]:
predictions[1]

array([0.9021458], dtype=float32)

In [ ]:
best_model.save("my_model")